In [1]:
import pandas as pd
import numpy as np

In [2]:
file = "purchase_data.csv"

In [3]:
purchase_data = pd.read_csv(file)

In [5]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [7]:
PlayerCount = purchase_data["SN"].nunique()
playercount_disp = pd.DataFrame({"Player Count": [PlayerCount]})
playercount_disp

,Player Count
0,576


In [8]:
unique_items = len(purchase_data["Item Name"].unique())
average_price = purchase_data["Price"].mean()
numpurchase = len(purchase_data["Item Name"])
revenue = purchase_data["Price"].sum()

Purchasing_Total = pd.DataFrame({"Number of Unique Items": [unique_items],
                                           "Average Price": [average_price],
                                           "Number of Purchases": [numpurchase],
                                           "Total Revenue": [revenue]})

Purchasing_Total

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [9]:
unique_players = purchase_data.drop_duplicates(["SN"],keep="first")

gender_count = pd.DataFrame(unique_players["Gender"].value_counts())
gender_count.columns = ["Total Count"]

total_users = gender_count["Total Count"].sum()
gender_count["Percentage"] = [round((x/total_users)*100) for x in gender_count["Total Count"]]

gender_count

,Total Count,Percentage
Male,484,84.0
Female,81,14.0
Other / Non-Disclosed,11,2.0


In [10]:
gender_purchase_total = purchase_data.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")
gender_average = purchase_data.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")
gender_counts = purchase_data.groupby(["Gender"]).count()["Price"].rename("Purchase Count")

normalized_total = gender_purchase_total / gender_count["Total Count"]

gender_data = pd.DataFrame({"Purchase Count": gender_counts, "Average Purchase Price": gender_average,
                            "Total Purchase Value": gender_purchase_total, "Normalized Totals": normalized_total})


gender_data["Average Purchase Price"] = gender_data["Average Purchase Price"]
gender_data["Total Purchase Value"] = gender_data["Total Purchase Value"]
gender_data ["Purchase Count"] = gender_data["Purchase Count"]
gender_data["Normalized Totals"] = gender_data["Normalized Totals"]
gender_data = gender_data.loc[:, ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]

gender_data

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other / Non-Disclosed,15,3.346000,50.19,4.562727


In [16]:
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

purchase_data["Age Group"] = pd.cut(purchase_data["Age"],age_bins, labels=group_names)
purchase_data

age_grouped = purchase_data.groupby("Age Group")

total_count_age = age_grouped["SN"].nunique()

percentage_by_age = (total_count_age/total_users) * 100

age_demographics = pd.DataFrame({"Percentage of Players": percentage_by_age, "Total Count": total_count_age})

age_demographics.index.name = None

age_demographics.style.format({"Percentage of Players":"{:,.2f}"})

,Percentage of Players,Total Count
<10,2.95,17
10-14,3.82,22
15-19,18.58,107
20-24,44.79,258
25-29,13.37,77
30-34,9.03,52
35-39,5.38,31
40+,2.08,12


In [17]:
purchase_data["Age Ranges"] = pd.cut(purchase_data["Age"], age_bins, labels=group_names)

age_purchase_total = purchase_data.groupby(["Age Ranges"]).sum()["Price"].rename("Total Purchase Value")
age_average = purchase_data.groupby(["Age Ranges"]).mean()["Price"].rename("Average Purchase Price")
age_counts = purchase_data.groupby(["Age Ranges"]).count()["Price"].rename("Purchase Count")

normalized_total = age_purchase_total / age_demographics["Total Count"]

age_data = pd.DataFrame({"Purchase Count": age_counts, "Average Purchase Price": age_average,
                         "Total Purchase Value": age_purchase_total, "Normalized Totals": normalized_total})

age_data

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Ranges,,,,
<10,23,3.353478,77.13,4.537059
10-14,28,2.956429,82.78,3.762727
15-19,136,3.035956,412.89,3.858785
20-24,365,3.052219,1114.06,4.318062
25-29,101,2.900990,293.00,3.805195
30-34,73,2.931507,214.00,4.115385
35-39,41,3.601707,147.67,4.763548
40+,13,2.941538,38.24,3.186667


In [19]:
user_total = purchase_data.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")
user_average = purchase_data.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
user_count = purchase_data.groupby(["SN"]).count()["Price"].rename("Purchase Count")

user_data = pd.DataFrame({"Total Purchase Value": user_total, "Average Purchase Price": user_average, "Purchase Count": user_count})

user_data.sort_values("Total Purchase Value", ascending=False).head(5)

,Total Purchase Value,Average Purchase Price,Purchase Count
SN,,,
Lisosia93,18.96,3.792000,5
Idastidru52,15.45,3.862500,4
Chamjask73,13.83,4.610000,3
Iral74,13.62,3.405000,4
Iskadarya95,13.10,4.366667,3


In [28]:
items = purchase_data[["Item ID", "Item Name", "Price"]]
 
item_stats = items.groupby(["Item ID","Item Name"])

purchase_count_item = item_stats["Price"].count()

purchase_value = (item_stats["Price"].sum()) 

item_price = purchase_value/purchase_count_item

most_popular_items = pd.DataFrame({"Purchase Count": purchase_count_item, 
                                   "Item Price": item_price,
                                   "Total Purchase Value":purchase_value})

popular_formatted = most_popular_items.sort_values(["Purchase Count"], ascending=False).head()

popular_formatted.style.format({"Item Price":"${:,.2f}",
                                "Total Purchase Value":"${:,.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [29]:
popular_formatted = most_popular_items.sort_values(["Total Purchase Value"],
                                                   ascending=False).head()

popular_formatted.style.format({"Item Price":"${:,.2f}",
                                "Total Purchase Value":"${:,.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
